<a href="https://colab.research.google.com/github/atlas-github/LPPEHScraper/blob/master/LPPEHScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'no-cache',
    'DNT': '1',
    'X-Requested-With': 'XMLHttpRequest',
    'X-MicrosoftAjax': 'Delta=true',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Mobile Safari/537.36',
    'Accept': '*/*',
    'Origin': 'http://search.lppeh.gov.my',
    'Referer': 'http://search.lppeh.gov.my/',
    'Accept-Language': 'en-US,en;q=0.9,ms;q=0.8',
}

In [0]:
compiled = []

for i in range(1000, 9999):
    data = {
      'ctl00$ContentPlaceHolder1$ToolkitScriptManager1': 'ctl00$ContentPlaceHolder1$UpdatePanel1',
      'ContentPlaceHolder1_ToolkitScriptManager1_HiddenField': '',
      '__EVENTTARGET': 'ctl00$ContentPlaceHolder1$btnSubmitMember',
      '__EVENTARGUMENT': '',
      '__VIEWSTATE': '/wEPDwUIOTI4MTg0NzIPFg4eCGlzTWVtYmVyaB4FUmVnTm9lHgRTa2lwZh4JUGFnZUNvdW50Zh4LQ3VycmVudFBhZ2UCAR4KU2VhcmNoVHlwZQIBHglGaWx0ZXJGb3JlFgJmD2QWAgIDD2QWAgIBD2QWAgIDD2QWAmYPZBYCAjcPZBYCAgEPZBYCZg9kFgQCCQ88KwAOAgAUKwACDxYIHhxFbmFibGVFbWJlZGRlZEJhc2VTdHlsZXNoZWV0Zx4XRW5hYmxlQWpheFNraW5SZW5kZXJpbmdoHhJSZXNvbHZlZFJlbmRlck1vZGULKXJUZWxlcmlrLldlYi5VSS5SZW5kZXJNb2RlLCBUZWxlcmlrLldlYi5VSSwgVmVyc2lvbj0yMDE1LjIuODI2LjQwLCBDdWx0dXJlPW5ldXRyYWwsIFB1YmxpY0tleVRva2VuPTEyMWZhZTc4MTY1YmEzZDQBHhVFbmFibGVFbWJlZGRlZFNjcmlwdHNnZBcCBQ9TZWxlY3RlZEluZGV4ZXMWAAULRWRpdEluZGV4ZXMWAAEWAhYLDwIDFCsAA2Q8KwAFAQQFCEZpcm1OYW1lPCsABQEEBQlOZWdvQ291bnRkZRQrAAALKXlUZWxlcmlrLldlYi5VSS5HcmlkQ2hpbGRMb2FkTW9kZSwgVGVsZXJpay5XZWIuVUksIFZlcnNpb249MjAxNS4yLjgyNi40MCwgQ3VsdHVyZT1uZXV0cmFsLCBQdWJsaWNLZXlUb2tlbj0xMjFmYWU3ODE2NWJhM2Q0ATwrAAcACyl0VGVsZXJpay5XZWIuVUkuR3JpZEVkaXRNb2RlLCBUZWxlcmlrLldlYi5VSSwgVmVyc2lvbj0yMDE1LjIuODI2LjQwLCBDdWx0dXJlPW5ldXRyYWwsIFB1YmxpY0tleVRva2VuPTEyMWZhZTc4MTY1YmEzZDQBFgIeBF9lZnMWAh4EX2VjY2RkZGRmZAILDzwrAA4CABQrAAIPFggfB2cfCGgfCQsrBAEfCmdkFwIFD1NlbGVjdGVkSW5kZXhlcxYABQtFZGl0SW5kZXhlcxYAARYCFgsPAgMUKwADZDwrAAUBBAUKTWVtYmVyTmFtZTwrAAUBBAUJTmVnb0NvdW50ZGUUKwAACysFATwrAAcACysGARYCHwsWAh8MZGRkZGZkGAEFHl9fQ29udHJvbHNSZXF1aXJlUG9zdEJhY2tLZXlfXxYCBSZjdGwwMCRDb250ZW50UGxhY2VIb2xkZXIxJGd2X0Zpcm0xTGlzdAUnY3RsMDAkQ29udGVudFBsYWNlSG9sZGVyMSRndl9NZW1iZXJMaXN0PjS9v60WnoplGouVdet7uAjbgX4In4YPC5m4sIsErI8=',
      '__VIEWSTATEGENERATOR': '656833E6',
      '__EVENTVALIDATION': '/wEdABwGWnMb8VLEn2sNPdUAAfxyTAB6IvnRsKjH/GIXkhJr230CjxlRlNH7aBuvsXQkT7eSlgEDS1Ec0Q5lEufrLzlmuhw3shugRLi/ezWMq/WOoW0ClP6J+KFBorrL6/IXBDaWc8jYeDPY8ZpB3+OoKMf2+RtfKPzAPx/CEPY7aSM2zVBzsyTU6xQXF2T6PNZdIRF4l20nDuzNoRHyV8B5Mlje7D+9DltTDOYsBc1Q0X2uUVcMtZ3McJC8PnxprqdN5ygcIl8jto5kwsGMr2BycnVwqvO1xt7gAlhiU2N/gF2JoNGYsl2HuTlzRWRUsWv+KO37hjtxtxE+Yk22OrUNR4QNbfvv2S257MRn4S6Mi259N2DCTycLq1FYOyIUUbcMFSq8Nx9UwBhEU3vEODmkiwkTA86wciyny2wAZTqLAoW/y3iFA5zaQUiPDKZgH918zLbkClnCsIZV5lhw33exYiUyOMltLyfcvtvCNKrWB7g36fc632YwPmF0ZKHRdI8H5S7ordAmCsB5fHjW5AkwtXI/w647KZyIMC/Zia3B8ua1mTDWHwg1Kuxd5yDqn5TxZAtqrqcqk9LIFhVWbG+1+xINY4JuO/Gryz/KSUaHjkymmfdapmGFpuYLW7nDv4BFV0E=',
      'ctl00$ContentPlaceHolder1$txt_FirmRegNo': '',
      'ctl00$ContentPlaceHolder1$txt_FirmName': '',
      'ctl00$ContentPlaceHolder1$txt_FirmCombineAdd': '',
      'ctl00$ContentPlaceHolder1$ddl_FirmType': '',
      'ctl00$ContentPlaceHolder1$ddl_StateDesc': '',
      'ctl00$ContentPlaceHolder1$txt_MemberRegNo': 'E'+str(i),
      'ctl00$ContentPlaceHolder1$txt_MemberName': '',
      'ctl00$ContentPlaceHolder1$ddl_LicenseType': '',
      'ctl00$ContentPlaceHolder1$txt_NegotiatorREN': '',
      'ctl00$ContentPlaceHolder1$txt_NegotiatorName': '',
      'ctl00$ContentPlaceHolder1$txt_NRIC_No': '',
      'ctl00$ContentPlaceHolder1$txt_PassportNo': '',
      'ctl00$ContentPlaceHolder1$txt_NegotiatorMobile': '',
      'ctl00$ContentPlaceHolder1$txt_modalFirm1Name': '',
      'ctl00_ContentPlaceHolder1_gv_Firm1List_ClientState': '',
      'ctl00_ContentPlaceHolder1_gv_MemberList_ClientState': '',
      '__ASYNCPOST': 'true',
      '': ''
    }

    response = requests.post('http://search.lppeh.gov.my/', headers=headers, data=data, verify=False)
    soup = BeautifulSoup(response.text, "html.parser")
    numbers = soup.findAll('strong')
    test = soup.findAll('p')
    try:
        row = list([numbers[0].get_text(), test[0].get_text(), test[1].get_text()])
    except:
        row = list([0, 0, 0])
    compiled.append(row)

In [0]:
compiled

In [0]:
phase1 = pd.DataFrame(compiled, columns = {"Member Number", "Member Name", "Company Name"})
phase1

In [0]:
phase1.to_csv('phase2.csv')
from google.cloud import files
files.download('phase2.csv')